In [ ]:
# Run once is enough!!
"""
##################################################################################
import os
import sys
import numpy as np
import random
import string
import pandas as pd

################ Paths and other configs - Set these #################################

data_dir = 'MultiNLI'
glue_dir = 'MultiNLI'

type_of_split = 'random'
assert type_of_split in ['preset', 'random']
# If 'preset', use the official train/val/test MultiNLI split
# If 'random', randomly split 50%/20%/30% of the data to train/val/test

######################################################################################

def tokenize(s):
    s = s.translate(str.maketrans('', '', string.punctuation))
    s = s.lower()
    s = s.split(' ')
    return s

### Read in data and assign train/val/test splits
train_df = pd.read_json(
    os.path.join(
        data_dir,
        'multinli_1.0_train.jsonl'),
    lines=True)

val_df = pd.read_json(
    os.path.join(
        data_dir,
        'multinli_1.0_dev_matched.jsonl'),
    lines=True)

test_df = pd.read_json(
    os.path.join(
        data_dir,
        'multinli_1.0_dev_mismatched.jsonl'),
    lines=True)

split_dict = {
    'train': 0,
    'val': 1,
    'test': 2
}

if type_of_split == 'preset':
    train_df['split'] = split_dict['train']
    val_df['split'] = split_dict['val']
    test_df['split'] = split_dict['test']
    df = pd.concat([train_df, val_df, test_df], ignore_index=True)

elif type_of_split == 'random':
    val_frac = 0.2
    test_frac = 0.3

    df = pd.concat([train_df, val_df, test_df], ignore_index=True)
    n = len(df)
    n_val = int(val_frac * n)
    n_test = int(test_frac * n)
    n_train = n - n_val - n_test
    splits = np.array([split_dict['train']] * n_train + [split_dict['val']] * n_val + [split_dict['test']] * n_test)
    np.random.shuffle(splits)
    df['split'] = splits

### Assign labels
df = df.loc[df['gold_label'] != '-', :]
print(f'Total number of examples: {len(df)}')
for k, v in split_dict.items():
    print(k, np.mean(df['split'] == v))

label_dict = {
    'contradiction': 0,
    'entailment': 1,
    'neutral': 2
}
for k, v in label_dict.items():
    idx = df.loc[:, 'gold_label'] == k
    df.loc[idx, 'gold_label'] = v

### Assign spurious attribute (negation words)
negation_words = ['nobody', 'no', 'never', 'nothing'] # Taken from https://arxiv.org/pdf/1803.02324.pdf

df['sentence2_has_negation'] = [False] * len(df)

for negation_word in negation_words:
    df['sentence2_has_negation'] |= [negation_word in tokenize(sentence) for sentence in df['sentence2']]

df['sentence2_has_negation'] = df['sentence2_has_negation'].astype(int)

## Write to disk
df = df[['gold_label', 'sentence2_has_negation', 'split']]
df.to_csv(os.path.join(data_dir, f'metadata_{type_of_split}.csv'))
print('Done')
"""

In [2]:
import os
import torch
import pandas as pd
from PIL import Image
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import Dataset, Subset    
import torch.nn as nn
import torch.nn.functional as F
    
    
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
class MultiNLIDataset():
    """
    MultiNLI dataset.
    label_dict = {
        'contradiction': 0,
        'entailment': 1,
        'neutral': 2
    }
    # Negation words taken from https://arxiv.org/pdf/1803.02324.pdf
    negation_words = ['nobody', 'no', 'never', 'nothing']
    """

    def __init__(self, root_dir,
                 target_name, confounder_names,
                 augment_data=False,
                 model_type=None):
        self.root_dir = root_dir
        self.target_name = target_name
        self.confounder_names = confounder_names
        self.model_type = model_type
        self.augment_data = augment_data

        assert len(confounder_names) == 1
        assert confounder_names[0] == 'sentence2_has_negation'
        assert target_name in ['gold_label_preset', 'gold_label_random']
        assert augment_data == False

        self.data_dir = os.path.join(
            self.root_dir)
        self.glue_dir = os.path.join(
            self.root_dir)
        if not os.path.exists(self.data_dir):
            raise ValueError(
                f'{self.data_dir} does not exist yet. Please generate the dataset first.')
        if not os.path.exists(self.glue_dir):
            raise ValueError(
                f'{self.glue_dir} does not exist yet. Please generate the dataset first.')

        # Read in metadata
        type_of_split = target_name.split('_')[-1]
        self.metadata_df = pd.read_csv(
            os.path.join(
                self.data_dir,
                f'metadata_{type_of_split}.csv'),
            index_col=0)

        # Get the y values
        # gold_label is hardcoded
        self.y_array = self.metadata_df['gold_label'].values
        self.n_classes = len(np.unique(self.y_array))

        self.confounder_array = self.metadata_df[confounder_names[0]].values
        print(np.sum(self.confounder_array==0))
        self.n_confounders = len(confounder_names)

        # Map to groups
        self.n_groups = len(np.unique(self.confounder_array)) * self.n_classes
        self.group_array = (self.y_array*(self.n_groups/self.n_classes) + self.confounder_array).astype('int')

        # Extract splits
        self.split_array = self.metadata_df['split'].values
        self.split_dict = {
            'train': 0,
            'val': 1,
            'test': 2
        }

        # Load features
        self.features_array = []
        for feature_file in [
            'cached_train_bert-base-uncased_128_mnli',  
            'cached_dev_bert-base-uncased_128_mnli',
            'cached_dev_bert-base-uncased_128_mnli-mm'
            ]:

            features = torch.load(
                os.path.join(
                    self.glue_dir,feature_file))

            self.features_array += features

        self.all_input_ids = torch.tensor([f.input_ids for f in self.features_array], dtype=torch.long)
        self.all_input_masks = torch.tensor([f.input_mask for f in self.features_array], dtype=torch.long)
        self.all_segment_ids = torch.tensor([f.segment_ids for f in self.features_array], dtype=torch.long)
        self.all_label_ids = torch.tensor([f.label_id for f in self.features_array], dtype=torch.long)

        self.x_array = torch.stack((
            self.all_input_ids,
            self.all_input_masks,
            self.all_segment_ids), dim=2)

        assert np.all(np.array(self.all_label_ids) == self.y_array)


    def __len__(self):
        return len(self.y_array)

    def __getitem__(self, idx):
        y = self.y_array[idx]
        a = self.confounder_array[idx]
        x = self.x_array[idx, ...]
        return x, y, a

    def group_str(self, group_idx):
        y = group_idx // (self.n_groups/self.n_classes)
        c = group_idx % (self.n_groups//self.n_classes)

        attr_name = self.confounder_names[0]
        group_name = f'{self.target_name} = {int(y)}, {attr_name} = {int(c)}'
        return group_name
    
data = MultiNLIDataset(root_dir = 'MultiNLI', target_name='gold_label_random',confounder_names=['sentence2_has_negation'])
train_indices = [idx for idx, split in enumerate(data.split_array) if split == 0]
training_set = torch.utils.data.Subset(data, train_indices)
val_indices = [idx for idx, split in enumerate(data.split_array) if split == 1]
vali_set = torch.utils.data.Subset(data, val_indices)
test_indices = [idx for idx, split in enumerate(data.split_array) if split == 2]
test_set = torch.utils.data.Subset(data, test_indices)

print('done')

/home/shenyu/miniconda3/envs/DLcourse/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


382945
done


In [3]:
batch_size = 100
training_data_loader = torch.utils.data.DataLoader(training_set, batch_size=batch_size, shuffle=True, drop_last=True)      
valid_data_loader = torch.utils.data.DataLoader(vali_set, batch_size=batch_size, shuffle=True, drop_last=False)      
test_data_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True, drop_last=False)      


In [4]:
print(len(training_set),len(vali_set),len(test_set))
k = iter(vali_set)
x,y,a = training_set[0]

206170 82466 123713


In [5]:
# test, don't run
x = iter(training_data_loader)
y = next(x)
a,b,c = y
print(a.shape)

from transformers import BertModel
bert_model = BertModel.from_pretrained("bert-base-uncased")

# Assuming your tensor 'a' is of shape [batch_size, sequence_length, 3]
input_ids = a[:,:,0] # Take all batch_size and sequence_length, only the 0th feature (token)
attention_mask = a[:,:,1] # Take all batch_size and sequence_length, only the 1st feature (attention)
token_type_ids = a[:,:,2] # Take all batch_size and sequence_length, only the 2nd feature (sequencebelong)

# Now pass these separate tensors into the model
out = bert_model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
print(out.last_hidden_state.shape)


torch.Size([100, 128, 3])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([100, 128, 768])


In [6]:
class MoCo(nn.Module):
    """
    Build a MoCo model with: a query encoder, a key encoder, and a queue
    Reference https://arxiv.org/abs/1911.05722
    """
    def __init__(self, base_encoder, dim=128, K=1500, m=0.999, T=0.07, mlp=False):
        super(MoCo, self).__init__()

        self.K = K
        self.m = m
        self.T = T

        # create the encoders
        self.encoder_q = base_encoder
        # create the queue
        self.register_buffer("queue", torch.randn(dim, K))
        self.queue = nn.functional.normalize(self.queue, dim=0)
        self.register_buffer("label_queue", torch.zeros(K).long())
        self.register_buffer("queue_ptr", torch.zeros(1, dtype=torch.long))

   
    @torch.no_grad()
    def _dequeue_and_enqueue(self, keys, labes):
        # gather keys before updating queue
        batch_size = keys.shape[0]
        ptr = int(self.queue_ptr)
        assert self.K % batch_size == 0  # for simplicity

        # replace the keys at ptr (dequeue and enqueue)
        self.queue[:, ptr:ptr + batch_size] = keys.T
        self.label_queue[ptr:ptr + batch_size] = labes
        ptr = (ptr + batch_size) % self.K  # move pointer

        self.queue_ptr[0] = ptr
  

    def forward(self, word, labels, training=True):
        # compute query features
        z, q = self.encoder_q(word, training=training)  # queries: NxC
        
        # dequeue and enqueue
        self._dequeue_and_enqueue(q, labels)
        self.queue = nn.functional.normalize(self.queue, dim=1)
        return self.queue, self.label_queue, z
    
    @torch.no_grad()
    def _inference(self,word, attention):
        with torch.no_grad():  # no gradient to keys
            z, q = self.encoder_q(word, training=False)  # queries: NxC
        return z


In [7]:

def choose_value_patch(atten, value, p_dim):
    # input insturction: 
    # atten: shape: Batch, Head, Patch
    # value: Batch, Head, Patch, Dim
    # Output: Batch, Head, Selct_Patch, dim
    atten = atten[:,:,1:]
    top_k_values, top_k_indices = torch.topk(atten, k=p_dim, dim=2, sorted=False)
    #top_k_indices : Batch, Head, Select_patch
    output = torch.gather(value, 2, top_k_indices.unsqueeze(-1).expand(-1,-1,-1,value.size(-1)))
    return output

class Last_Attention(nn.Module):
    def __init__(self, type_id):
        super(Last_Attention, self).__init__()
        if type_id == 'base':
            self.emb_size = 768
        if type_id == 'large':
            self.emb_size = 1024
            
        self.p_dim = 2
        self.head = 8
        self.temperature = 1
        self.head_dim = self.emb_size //self.head
        self.Q = nn.Linear(self.emb_size,self.emb_size)
        self.K = nn.Linear(self.emb_size,self.emb_size)
        self.V = nn.Linear(self.emb_size,self.emb_size)
        self.projection = nn.Linear(self.emb_size, self.emb_size)
        self.soft_max = nn.Softmax(dim=-1)
        self.projector = nn.Sequential(
            nn.Linear(self.p_dim*self.emb_size, self.emb_size, bias=False),
            nn.ReLU(),
            nn.Linear(self.emb_size, 128, bias=False),
        )
        self.cp = True
        self.momentum = 0.1
        if type_id == 'large':
            self.register_buffer('running_mean_q', torch.zeros(1,self.head,128,128))
            self.register_buffer('running_std_q', torch.ones(1,self.head,128,128))
            self.register_buffer('running_mean_k', torch.zeros(1,self.head,128,128))
            self.register_buffer('running_std_k', torch.ones(1,self.head,128,128))
        if type_id == 'base':
            self.register_buffer('running_mean_q', torch.zeros(1,self.head,128,96))
            self.register_buffer('running_std_q', torch.ones(1,self.head,128,96))
            self.register_buffer('running_mean_k', torch.zeros(1,self.head,128,96))
            self.register_buffer('running_std_k', torch.ones(1,self.head,128,96))
    #1, 8, 160, 96  qshape torch.Size([100, 8, 128, 96])
    def register_buffer(self, name, tensor):
        setattr(self, name, tensor)
        
    def forward(self, x, training):
        B, N, C = x.shape
        origin_k = self.K(x)
        origin_q = self.Q(x)
        origin_v = self.V(x)
        
        q = origin_q.reshape(B,N,self.head, C//self.head).permute(0,2,1,3)
        k = origin_k.reshape(B,N,self.head, C//self.head).permute(0,2,1,3)
        
        self.running_mean_q = self.running_mean_q.detach()
        self.running_std_q = self.running_std_q.detach()
        self.running_mean_k = self.running_mean_k.detach()
        self.running_std_k = self.running_std_k.detach()

        
        if training:
            q_mean, q_std = torch.mean(q, 0, keepdim=True), torch.std(q, 0, keepdim=True)
            k_mean, k_std = torch.mean(k, 0, keepdim=True), torch.std(k, 0, keepdim=True) 

            self.running_mean_q = (1 - self.momentum) * self.running_mean_q.to(device) + self.momentum * q_mean
            self.running_std_q = (1 - self.momentum) * self.running_std_q.to(device) + self.momentum * q_std
            self.running_mean_k = (1 - self.momentum) * self.running_mean_k.to(device) + self.momentum * k_mean
            self.running_std_k = (1 - self.momentum) * self.running_std_k.to(device) + self.momentum * k_std
        else:
            q_mean = self.running_mean_q
            q_std = self.running_std_q
            k_mean = self.running_mean_k
            k_std = self.running_std_k
        
        q = (q - q_mean) /q_std
        k = (k - k_mean) /k_std
        
        v = origin_v.reshape(B,N,self.head, C//self.head).permute(0,2,1,3)
        attention = (q @ k.transpose(-2,-1))* (self.head_dim ** (-0.5))        
        atten = self.soft_max(attention/self.temperature)
        out = (atten @ v).transpose(1, 2).reshape(B, N, C)
        out = self.projection(out)
        
        #attentions = atten.permute(0, 2, 1, 3) # Batch, Patch, Head, Patch
        attentions = atten[:,:, 0, :]
        
        #fairness process
        mst_val = choose_value_patch(attentions, v, self.p_dim)
        mst_val = mst_val.reshape(B, -1)
        mst_val = self.projector(mst_val)
        z = F.normalize(mst_val)

        return out, z

    
class Last_ATBlock(nn.Module):
    def __init__(self, type_id):
        super().__init__()
        if type_id == 'base':
            dim = 768
        if type_id == 'large':
            dim = 1024
        self.norm = nn.LayerNorm(dim)
        self.attention = Last_Attention(type_id)
        self.norm2 = nn.LayerNorm(dim)
        self.feedforward = nn.Sequential(
            nn.Linear(dim, dim),
            nn.ReLU(),
            nn.Linear(dim, dim)          
        )
        
    def forward(self, x, training):
        identity = x
        x = self.norm(x)
        x, vz = self.attention(x, training)
        x += identity
        res = x 
        x = self.norm2(x)
        x = self.feedforward(x)
        x += res
        return x, vz

    
class BERT_base(nn.Module):
    def __init__(self, BERT, type_id):
        super(BERT_base, self).__init__()
        if type_id == 'base':
            p_dim = 768
        if type_id == 'large':
            p_dim = 1024
        self.BERT = BERT
        self.last_layer = Last_ATBlock(type_id)
        self.seq = nn.Sequential(
            nn.Linear(p_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 3)
        )

    def forward(self, word, training):
        input_ids = word[:,:,0] # (token)
        attention_mask = word[:,:,1] # (attention)
        token_type_ids = word[:,:,2] # (sequencebelong)
        
        x = self.BERT(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        x = x.last_hidden_state
        hidden, v = self.last_layer(x, training)
        y = self.seq(hidden[:,0])
        return y, v


In [8]:
import random
import numpy as np
from sklearn.metrics import accuracy_score, precision_score
from sklearn.metrics import confusion_matrix
from collections import OrderedDict
import seaborn as sns
import torch.optim as optim
from tqdm import tqdm
from transformers import BertModel

def compute_fairness(cf1, cf2):
    dp = []
    TPR = []
    FPR = []
    for cf in (cf1, cf2):
        TP = np.diag(cf)
        FN = cf.sum(axis =1)-np.diag(cf)
        FP = cf.sum(axis = 0) - np.diag(cf)
        TN = cf.sum()-(FN+FP+TP)

        dp_value = (TP+FP)/(TN+FP+FN+TP)
        TPR_value = TP/(TP+FN)
        FPR_value = FP/(FP+TN)
        dp.append(dp_value)
        TPR.append(TPR_value)
        FPR.append(FPR_value)
    DP = abs(dp[0]-dp[1])
    EoP = abs(TPR[0] - TPR[1])
    EoD = 0.5*(abs(FPR[0]-FPR[1])+abs(TPR[0]-TPR[1]))
    return DP, EoP, EoD  

class SupervisedContrastiveLoss_v2(nn.Module):
    def __init__(self, temperature=0.07):
        """
        Implementation of the loss described in the paper Supervised Contrastive Learning :
        https://arxiv.org/abs/2004.11362
        :param temperature: int
        """
        super(SupervisedContrastiveLoss_v2, self).__init__()
        self.temperature = temperature

    def forward(self, projections, targets):
        dot_product_tempered = torch.mm(projections, projections.T) / self.temperature
        exp_dot_tempered = (
            torch.exp(dot_product_tempered - torch.max(dot_product_tempered, dim=1, keepdim=True)[0]) + 1e-5
        )

        mask_similar_class = (targets.unsqueeze(1).repeat(1, targets.shape[0]) == targets).to(device)
        mask_anchor_out = (1 - torch.eye(exp_dot_tempered.shape[0])).to(device)
        mask_combined = mask_similar_class * mask_anchor_out
        cardinality_per_samples = torch.sum(mask_combined, dim=1)

        log_prob = -torch.log(exp_dot_tempered / (torch.sum(exp_dot_tempered * mask_anchor_out, dim=1, keepdim=True)))
        supervised_contrastive_loss_per_sample = torch.sum(log_prob * mask_combined, dim=1) / cardinality_per_samples
        supervised_contrastive_loss = torch.mean(supervised_contrastive_loss_per_sample)

        return supervised_contrastive_loss
    
def seed_everything(seed):
    """
    Changes the seed for reproducibility. 
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


#x,y,a
def train_model(type_id):
    epoch = 5
    if type_id == 'base':
        BERT = BertModel.from_pretrained("bert-base-uncased")
        lr = 5e-5
    if type_id =='large':
        BERT = BertModel.from_pretrained("bert-large-uncased")
        lr = 0.5e-5
        
    model = BERT_base(BERT, type_id).to(device)
    model_moco = MoCo(model).to(device)
    
    criterion = nn.CrossEntropyLoss()
    acc = 0
    fair_criterion = SupervisedContrastiveLoss_v2()
    optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model_moco.parameters()), lr=lr)
    
    for epoches in range(epoch):
        
        with tqdm(training_data_loader, unit="batch") as tepoch:
            model.train()
            model_moco.train()
            for word, train_target, _ in tepoch:
                word = word.to(device)
                train_target = train_target.to(device)
                train_target_onehot = torch.nn.functional.one_hot(train_target, num_classes=3)
                train_target_onehot = train_target_onehot.float().to(device)
                optimizer.zero_grad()
                
                features, targets, outputs = model_moco(word, train_target, training=True)
                fair_loss = fair_criterion(features.T, targets)
                ut_loss = criterion(outputs, train_target_onehot)
                loss =  ut_loss + 0.8*fair_loss
                tepoch.set_postfix(u= ut_loss.item(), f= fair_loss.item()) 
                loss.backward()
                optimizer.step()
                tepoch.set_description(f"epoch %2f " % epoches)
                
                
          
        model.eval()
        model_moco.eval()
        test_pred = []
        test_gt = []
        sense_gt = []
        female_predic = []
        female_gt = []
        male_predic = []
        male_gt = []
    # Evaluate on test set.
        for step, (test_word, test_target, test_sensitive) in enumerate(test_data_loader):
            test_word = test_word.to(device)
            test_target = test_target.to(device)
            
            gt = test_target.detach().cpu().numpy()
            sen = test_sensitive.detach().cpu().numpy()
            test_gt.extend(gt)
            sense_gt.extend(sen)

            with torch.no_grad():
                prediction,_ = model(test_word, training=False)
                test_pred_ = torch.argmax(prediction, dim=1)
                test_pred.extend(test_pred_.detach().cpu().numpy())

        for i in range(len(sense_gt)):
            if sense_gt[i] == 0:
                female_predic.append(test_pred[i])
                female_gt.append(test_gt[i])
            else:
                male_predic.append(test_pred[i])
                male_gt.append(test_gt[i])
        female_CM = confusion_matrix(female_gt, female_predic) 
        male_CM = confusion_matrix(male_gt, male_predic) 
        DP, EoP, EoD = compute_fairness(female_CM , male_CM)
        ACC = accuracy_score(test_gt, test_pred)
        print(female_CM)
        print('acc:', ACC)
        print('DP:', max(DP))
        print('EoP' , max(EoP))
        print('EoD', max(EoD))
        print('Trade off', ACC *(1-max(EoD)))
        
        
seed_everything(1024)    
train_model('base')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
epoch 0.000000 : 100%|██████████████████████████████████████████| 2061/2061 [12:00<00:00,  2.86batch/s, f=6.93, u=0.401]


[[27272  2736  4656]
 [ 1481 34806  4071]
 [ 3459  4577 31818]]
acc: 0.8230016247282016
DP: 0.5140936966644006
EoP 0.23118715795647304
EoD 0.17715169391144586
Trade off 0.6772054928157285


epoch 1.000000 : 100%|██████████████████████████████████████████| 2061/2061 [11:56<00:00,  2.88batch/s, f=6.89, u=0.416]


[[28791  1988  3885]
 [ 1995 34233  4130]
 [ 4286  4433 31135]]
acc: 0.82525684447067
DP: 0.4822944915560694
EoP 0.18258702085742207
EoD 0.134722739557246
Trade off 0.7140759815452133


epoch 2.000000 : 100%|██████████████████████████████████████████| 2061/2061 [11:56<00:00,  2.87batch/s, f=6.86, u=0.205]


[[28636  1765  4263]
 [ 2025 32947  5386]
 [ 4209  3773 31872]]
acc: 0.8194611722292726
DP: 0.4879003688733704
EoP 0.20363053888188487
EoD 0.14726130509209204
Trade off 0.6987862505344944


epoch 3.000000 : 100%|██████████████████████████████████████████| 2061/2061 [11:54<00:00,  2.88batch/s, f=6.83, u=0.238]


[[28837  1695  4132]
 [ 2381 32812  5165]
 [ 4502  3852 31500]]
acc: 0.8170362047642528
DP: 0.4733719699526305
EoP 0.15688150354235142
EoD 0.12527439340863622
Trade off 0.7146824898195167


epoch 4.000000 : 100%|██████████████████████████████████████████| 2061/2061 [11:55<00:00,  2.88batch/s, f=6.83, u=0.149]


[[29562  1659  3443]
 [ 2829 32876  4653]
 [ 5580  4281 29993]]
acc: 0.810787871929385
DP: 0.4475530967254819
EoP 0.12587120711686095
EoD 0.09972004017233231
Trade off 0.7299360727693469
